<a href="https://colab.research.google.com/github/MaryamNourii/Intent-detection-and-Slot-filling/blob/main/BERT_intent_Slot_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers
!pip install nlpaug

In [3]:
!unzip -qx dataset.zip -d dataset

In [4]:
# MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
# MODEL_NAME_OR_PATH = "bert-base-multilingual-uncased"
MODEL_NAME_OR_PATH =  "HooshvareLab/bert-fa-zwnj-base"

In [5]:
import pandas as pd
import nlpaug.augmenter.word as naw

data = pd.read_csv("/content/dataset/train.csv")

def augment_data(sentence, slots, intent_label, aug):
    augmented_sentence = aug.augment(sentence)
    return augmented_sentence, slots, intent_label

aug = naw.ContextualWordEmbsAug(model_path=MODEL_NAME_OR_PATH, action="insert")

aug_data = []
for i in range(len(data)):
    sentence = data.loc[i, 'sentence']
    slots = data.loc[i, 'slots']
    intent_label = data.loc[i, 'intent_label']
    augmented_sentence, slots, intent_label = augment_data(sentence, slots, intent_label, aug)
    aug_data.append([''.join(augmented_sentence), slots, intent_label])

aug_data = pd.DataFrame(aug_data, columns=['sentence', 'slots', 'intent_label'])

augmented_data = pd.concat([data, aug_data], ignore_index=True)
augmented_data=augmented_data.drop('index',axis=1)
augmented_data = augmented_data.sample(frac=1).reset_index(drop=True)
augmented_data.to_csv('/content/dataset/train.csv', index=False)